# Connection semantics prototype

## Motivation and concepts
Typically one would create connections in NEST by calling `Connect()` on two NodeCollections. However, every time `Connect()` is called, it has to go from PyNEST, through SLI and into the C++ kernel, iterate over connections, before it returns to PyNEST. One may also use multiple threads in the iteration over connections, where initiating and ending a parallel region causes an additional overhead. Therefore, if a simulation involves many `Connect()` calls, this entails a significant overhead in the connection stage.

To reduce the overhead of repeated calls to `Connect()`, we introduce projections. With projections, instead of immediately creating the connections, the connection specifications are stored until the user specifies that the connections should be made, typically after specifying all projections. That way we can have only a single push from PyNEST through SLI to the C++ kernel, and reduce the number of times we enter the parallel region.

## How it works
The projections are introduced as objects based on existing connection rules, such as `OneToOne` from the `one_to_one` rule, or `FixedIndegree` from the `fixed_indegree` rule. The projection object is then added to a connection buffer on the PyNEST level, using `Connect()`. When all projections have been defined and added to the buffer, the user can call `BuildNetwork()`, which sends all projections in a single package to the C++ level. Once on the C++ level, the projections that have the same connection rule can be combined automatically, reducing the number of times we enter the parallel region when connecting.

## What is currently implemented

- A PyNEST interface with several projection classes, implemented in the top level of `nest`.
- `nest.Connect()` now only works with projection objects.
- Projections work with both normal connections, and spatially structured connections involving for example distance based probabilities or masks.
- Classes for all synapse models, in the submodule `nest.synapsemodels`. The synapse object created can be used as the synapse specification when creating projections.
- Synapse objects can be combined with `CollocatedSynapses`, and used instead of a single synapse model object when creating a projection.
- Basic C++ implementation for connection, without any processing of the projections before connecting.

## Remaining work and next steps

- Documentation
- Smarter C++ implementation

In [1]:
import nest

## Projection object

Introducing the Projection object, representing projection between two populations with a certain rule. The projection object is added to a buffer with `Connect()`, and held until calling `BuildNetwork()`.

### `OneToOne`

In [2]:
nest.ResetKernel()
n = nest.Create('iaf_psc_alpha', 5)

projection = nest.OneToOne(source=n, target=n)
nest.Connect(projection)
nest.BuildNetwork()

conns = nest.GetConnections()
print(conns)

Connecting 1 projections...
 source   target   synapse model   weight   delay 
-------- -------- --------------- -------- -------
      1        1  static_synapse    1.000   1.000
      2        2  static_synapse    1.000   1.000
      3        3  static_synapse    1.000   1.000
      4        4  static_synapse    1.000   1.000
      5        5  static_synapse    1.000   1.000


### `FixedIndegree`
Different Projections can take different arguments, depending on the connection rule.

In [3]:
nest.ResetKernel()
n = nest.Create('iaf_psc_alpha', 5)

projection = nest.FixedIndegree(source=n, target=n, indegree=5)
nest.Connect(projection)
nest.BuildNetwork()

conns = nest.GetConnections()
print(conns)

Connecting 1 projections...
 source   target   synapse model   weight   delay 
-------- -------- --------------- -------- -------
      1        2  static_synapse    1.000   1.000
      1        2  static_synapse    1.000   1.000
      1        5  static_synapse    1.000   1.000
      1        3  static_synapse    1.000   1.000
      1        5  static_synapse    1.000   1.000
      2        5  static_synapse    1.000   1.000
      2        4  static_synapse    1.000   1.000
      3        1  static_synapse    1.000   1.000
      3        1  static_synapse    1.000   1.000
      3        2  static_synapse    1.000   1.000
      3        3  static_synapse    1.000   1.000
      3        3  static_synapse    1.000   1.000
      3        4  static_synapse    1.000   1.000
      4        5  static_synapse    1.000   1.000
      4        5  static_synapse    1.000   1.000
      4        1  static_synapse    1.000   1.000
      4        2  static_synapse    1.000   1.000
      4        2  st

### `FixedOutdegree`

In [4]:
nest.ResetKernel()
n = nest.Create('iaf_psc_alpha', 5)

projection = nest.FixedOutdegree(source=n, target=n, outdegree=5)
nest.Connect(projection)
nest.BuildNetwork()

conns = nest.GetConnections()
print(conns)

Connecting 1 projections...
 source   target   synapse model   weight   delay 
-------- -------- --------------- -------- -------
      1        1  static_synapse    1.000   1.000
      1        4  static_synapse    1.000   1.000
      1        5  static_synapse    1.000   1.000
      1        5  static_synapse    1.000   1.000
      1        3  static_synapse    1.000   1.000
      2        5  static_synapse    1.000   1.000
      2        4  static_synapse    1.000   1.000
      2        3  static_synapse    1.000   1.000
      2        2  static_synapse    1.000   1.000
      2        4  static_synapse    1.000   1.000
      3        4  static_synapse    1.000   1.000
      3        3  static_synapse    1.000   1.000
      3        3  static_synapse    1.000   1.000
      3        1  static_synapse    1.000   1.000
      3        1  static_synapse    1.000   1.000
      4        5  static_synapse    1.000   1.000
      4        4  static_synapse    1.000   1.000
      4        3  st

### `PairwiseBernoulli`

In [5]:
nest.ResetKernel()
n = nest.Create('iaf_psc_alpha', 10)

projection = nest.PairwiseBernoulli(source=n, target=n, p=0.2)
nest.Connect(projection)
nest.BuildNetwork()

conns = nest.GetConnections()
print(conns)

Connecting 1 projections...
 source   target   synapse model   weight   delay 
-------- -------- --------------- -------- -------
      1        7  static_synapse    1.000   1.000
      2        3  static_synapse    1.000   1.000
      3        5  static_synapse    1.000   1.000
      5        2  static_synapse    1.000   1.000
      5        3  static_synapse    1.000   1.000
      5        8  static_synapse    1.000   1.000
      6        7  static_synapse    1.000   1.000
      7        1  static_synapse    1.000   1.000
      7        4  static_synapse    1.000   1.000
      7        5  static_synapse    1.000   1.000
      7        9  static_synapse    1.000   1.000
      8        7  static_synapse    1.000   1.000
      8        8  static_synapse    1.000   1.000
      9        1  static_synapse    1.000   1.000
      9        6  static_synapse    1.000   1.000
      9        7  static_synapse    1.000   1.000
     10        3  static_synapse    1.000   1.000


### `FixedTotalNumber`
Can also pass arguments that controls if autapses or multapses are possible.

In [6]:
nest.ResetKernel()
n = nest.Create('iaf_psc_alpha', 5)

projection = nest.FixedTotalNumber(source=n, target=n, N=10, 
                                   allow_autapses=True,
                                   allow_multapses=True)
nest.Connect(projection)
nest.BuildNetwork()

conns = nest.GetConnections()
print(conns)

Connecting 1 projections...
 source   target   synapse model   weight   delay 
-------- -------- --------------- -------- -------
      1        4  static_synapse    1.000   1.000
      1        3  static_synapse    1.000   1.000
      1        4  static_synapse    1.000   1.000
      2        3  static_synapse    1.000   1.000
      3        4  static_synapse    1.000   1.000
      4        3  static_synapse    1.000   1.000
      5        5  static_synapse    1.000   1.000
      5        3  static_synapse    1.000   1.000
      5        3  static_synapse    1.000   1.000
      5        4  static_synapse    1.000   1.000


## Multiple projections
Multiple Projections can be added to the buffer before connecting, by calling `Connect()` multiple times.

In [7]:
nest.ResetKernel()

N = 10
IN_A = 2
IN_B = 5
n = nest.Create('iaf_psc_alpha', N)
nest.Connect(nest.FixedIndegree(source=n, target=n, indegree=IN_A))
nest.Connect(nest.FixedIndegree(source=n, target=n, indegree=IN_B))
nest.BuildNetwork()

conns = nest.GetConnections()
print(conns)

Connecting 2 projections...
 source   target   synapse model   weight   delay 
-------- -------- --------------- -------- -------
      1       10  static_synapse    1.000   1.000
      1        4  static_synapse    1.000   1.000
      1        1  static_synapse    1.000   1.000
      1        4  static_synapse    1.000   1.000
      1        5  static_synapse    1.000   1.000
      1        8  static_synapse    1.000   1.000
      1        9  static_synapse    1.000   1.000
      1        2  static_synapse    1.000   1.000
      1        5  static_synapse    1.000   1.000
      2       10  static_synapse    1.000   1.000
      2       10  static_synapse    1.000   1.000
      2        6  static_synapse    1.000   1.000
      2        8  static_synapse    1.000   1.000
      2        1  static_synapse    1.000   1.000
      3        1  static_synapse    1.000   1.000
     ⋮        ⋮               ⋮        ⋮       ⋮ 
      8        1  static_synapse    1.000   1.000
      8        8  st

Or by passing a list of Projections.

In [8]:
nest.ResetKernel()

N = 10
IN_A = 2
IN_B = 5
n = nest.Create('iaf_psc_alpha', N)
projections = [nest.FixedIndegree(source=n, target=n, indegree=IN_A),
               nest.FixedIndegree(source=n, target=n, indegree=IN_B)]
nest.Connect(projections)
nest.BuildNetwork()

conns = nest.GetConnections()
print(conns)

Connecting 2 projections...
 source   target   synapse model   weight   delay 
-------- -------- --------------- -------- -------
      1       10  static_synapse    1.000   1.000
      1        4  static_synapse    1.000   1.000
      1        1  static_synapse    1.000   1.000
      1        4  static_synapse    1.000   1.000
      1        5  static_synapse    1.000   1.000
      1        8  static_synapse    1.000   1.000
      1        9  static_synapse    1.000   1.000
      1        2  static_synapse    1.000   1.000
      1        5  static_synapse    1.000   1.000
      2       10  static_synapse    1.000   1.000
      2       10  static_synapse    1.000   1.000
      2        6  static_synapse    1.000   1.000
      2        8  static_synapse    1.000   1.000
      2        1  static_synapse    1.000   1.000
      3        1  static_synapse    1.000   1.000
     ⋮        ⋮               ⋮        ⋮       ⋮ 
      8        1  static_synapse    1.000   1.000
      8        8  st

## Synapse model object
We introduce synapse model objects, which hold information previously passed as a `dict` to `syn_spec`.

### `static_synapse`

In [9]:
nest.ResetKernel()
n = nest.Create('iaf_psc_alpha', 5)

synapse = nest.synapsemodels.static(weight=0.5, delay=0.7)

projection = nest.OneToOne(source=n, target=n, syn_spec=synapse)
nest.Connect(projection)
nest.BuildNetwork()

conns = nest.GetConnections()
print(conns)

Connecting 1 projections...
 source   target   synapse model   weight   delay 
-------- -------- --------------- -------- -------
      1        1  static_synapse   0.5000  0.7000
      2        2  static_synapse   0.5000  0.7000
      3        3  static_synapse   0.5000  0.7000
      4        4  static_synapse   0.5000  0.7000
      5        5  static_synapse   0.5000  0.7000


### `stdp_synapse`

In [10]:
nest.ResetKernel()
n = nest.Create('iaf_psc_alpha', 5)

synapse = nest.synapsemodels.stdp(weight=nest.random.uniform(0.5, 1.0), tau_plus=17.)

projection = nest.OneToOne(source=n, target=n, syn_spec=synapse)
nest.Connect(projection)
nest.BuildNetwork()

conns = nest.GetConnections()
print(conns)
print('tau_plus =', conns.tau_plus)

Connecting 1 projections...
 source   target   synapse model   weight   delay 
-------- -------- --------------- -------- -------
      1        1    stdp_synapse   0.9014   1.000
      2        2    stdp_synapse   0.9508   1.000
      3        3    stdp_synapse   0.8267   1.000
      4        4    stdp_synapse   0.7670   1.000
      5        5    stdp_synapse   0.7699   1.000
tau_plus = [17.0, 17.0, 17.0, 17.0, 17.0]


## Usage examples

### Generate projections iteratively, with duplicated projections
To iteratively modify a Projection, ideally wrap a projection with `itertools.partial`.

In [11]:
import itertools
from functools import partial

nest.ResetKernel()
n1 = nest.Create('iaf_psc_alpha', 3)
n2 = nest.Create('iaf_psc_exp', 3)

synapse = nest.synapsemodels.static(weight=0.5, delay=0.7)
base_projection = nest.FixedTotalNumber(N=5, syn_spec=synapse)
create_projection = partial(nest.FixedTotalNumber, N=5, syn_spec=synapse) # create prototype

for source, target in itertools.product((n1, n2), repeat=2): # itertools.product to map out all combinations
    projection = create_projection(source=source, target=target) # modify a few parameters each time
    nest.Connect(projection)

nest.BuildNetwork()

conns = nest.GetConnections()
print(conns)
print('Number of connections:', len(conns))

Connecting 4 projections...
 source   target   synapse model   weight   delay 
-------- -------- --------------- -------- -------
      1        6  static_synapse   0.5000  0.7000
      1        5  static_synapse   0.5000  0.7000
      1        3  static_synapse   0.5000  0.7000
      1        2  static_synapse   0.5000  0.7000
      2        5  static_synapse   0.5000  0.7000
      2        6  static_synapse   0.5000  0.7000
      3        6  static_synapse   0.5000  0.7000
      3        2  static_synapse   0.5000  0.7000
      3        2  static_synapse   0.5000  0.7000
      3        2  static_synapse   0.5000  0.7000
      4        6  static_synapse   0.5000  0.7000
      4        6  static_synapse   0.5000  0.7000
      4        2  static_synapse   0.5000  0.7000
      5        5  static_synapse   0.5000  0.7000
      5        4  static_synapse   0.5000  0.7000
      5        3  static_synapse   0.5000  0.7000
      5        3  static_synapse   0.5000  0.7000
      6        4  st

### Using `CollocatedSynapses` in projections
Projections support passing a `CollocatedSynapses` object as `syn_spec`. With `CollocatedSynapses`, connections with different synapse parameters can be created between the same nodes.

In [12]:
nest.ResetKernel()
n = nest.Create('iaf_psc_alpha', 2)

syn_spec = nest.CollocatedSynapses(nest.synapsemodels.static(weight=-2.),
                                   nest.synapsemodels.stdp(weight=3.))

projection = nest.FixedIndegree(source=n, target=n, indegree=3, syn_spec=syn_spec)
nest.Connect(projection)
nest.BuildNetwork()

conns = nest.GetConnections()
print(conns)

Connecting 1 projections...
 source   target   synapse model   weight   delay 
-------- -------- --------------- -------- -------
      2        1  static_synapse   -2.000   1.000
      2        1  static_synapse   -2.000   1.000
      2        1  static_synapse   -2.000   1.000
      2        2  static_synapse   -2.000   1.000
      2        2  static_synapse   -2.000   1.000
      2        2  static_synapse   -2.000   1.000
      2        1    stdp_synapse    3.000   1.000
      2        1    stdp_synapse    3.000   1.000
      2        1    stdp_synapse    3.000   1.000
      2        2    stdp_synapse    3.000   1.000
      2        2    stdp_synapse    3.000   1.000
      2        2    stdp_synapse    3.000   1.000


### Spatial connections
Projections for spatial connections are created the same way as regular connections.

In [13]:
nest.ResetKernel()

dim = [4, 5]
extent = [10., 10.]
layer = nest.Create('iaf_psc_alpha', positions=nest.spatial.grid(dim, extent=extent))

mask = {'rectangular': {
        'lower_left': [-5., -5.],
        'upper_right': [0., 0.]}}
projection = nest.FixedIndegree(source=layer, target=layer, indegree=1, mask=mask)

nest.Connect(projection)
nest.BuildNetwork()

conns = nest.GetConnections()
print(conns)

Connecting 1 projections...
 source   target   synapse model   weight   delay 
-------- -------- --------------- -------- -------
      2        7  static_synapse    1.000   1.000
      3        1  static_synapse    1.000   1.000
      4        2  static_synapse    1.000   1.000
      4        3  static_synapse    1.000   1.000
      4        9  static_synapse    1.000   1.000
      5        4  static_synapse    1.000   1.000
      5        5  static_synapse    1.000   1.000
      5       15  static_synapse    1.000   1.000
      7        6  static_synapse    1.000   1.000
      8       12  static_synapse    1.000   1.000
      9        8  static_synapse    1.000   1.000
     10       10  static_synapse    1.000   1.000
     10       14  static_synapse    1.000   1.000
     10       19  static_synapse    1.000   1.000
     12       11  static_synapse    1.000   1.000
     14       13  static_synapse    1.000   1.000
     15       18  static_synapse    1.000   1.000
     15       20  st

### Spatial connections with `CollocatedSynapses`

In [14]:
nest.ResetKernel()

dim = [4, 2]
extent = [10., 10.]
layer = nest.Create('iaf_psc_alpha', positions=nest.spatial.grid(dim, extent=extent))

mask = {'rectangular': {
        'lower_left': [-5., -5.],
        'upper_right': [0., 0.]}}

syn_spec = nest.CollocatedSynapses(nest.synapsemodels.static(weight=-2.),
                                   nest.synapsemodels.stdp(weight=3.))

projection = nest.FixedIndegree(source=layer, target=layer, indegree=1, syn_spec=syn_spec, mask=mask)

nest.Connect(projection)
nest.BuildNetwork()

conns = nest.GetConnections()
print(conns)

Connecting 1 projections...
 source   target   synapse model   weight   delay 
-------- -------- --------------- -------- -------
      2        1  static_synapse   -2.000   1.000
      2        2  static_synapse   -2.000   1.000
      3        3  static_synapse   -2.000   1.000
      3        7  static_synapse   -2.000   1.000
      4        4  static_synapse   -2.000   1.000
      4        5  static_synapse   -2.000   1.000
      6        6  static_synapse   -2.000   1.000
      8        8  static_synapse   -2.000   1.000
      2        1    stdp_synapse    3.000   1.000
      2        2    stdp_synapse    3.000   1.000
      3        3    stdp_synapse    3.000   1.000
      3        7    stdp_synapse    3.000   1.000
      4        4    stdp_synapse    3.000   1.000
      4        5    stdp_synapse    3.000   1.000
      6        6    stdp_synapse    3.000   1.000
      8        8    stdp_synapse    3.000   1.000
